# Read in the Data

In [1]:
# import libraries needed
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [2]:
# create spark session
spark = SparkSession.builder.appName("FinalProject").getOrCreate()

In [3]:
# data : https://learn.microsoft.com/en-us/azure/open-datasets/dataset-taxi-yellow?tabs=pyspark#azure-databricks

# Azure storage access info
blob_account_name = "azureopendatastorage"
blob_container_name = "nyctlc"
blob_relative_path = "yellow"
blob_sas_token = "r"

# Allow SPARK to read from Blob remotely
wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path)
spark.conf.set(
  'fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name),
  blob_sas_token)
print('Remote blob path: ' + wasbs_path)

# SPARK read parquet, note that it won't load any data yet by now
df = spark.read.parquet(wasbs_path)
print('Register the DataFrame as a SQL temporary view: source')
df.createOrReplaceTempView('source')

# Display top 10 rows
print('Displaying top 10 rows: ')
display(spark.sql('SELECT * FROM source LIMIT 10'))

Remote blob path: wasbs://nyctlc@azureopendatastorage.blob.core.windows.net/yellow
Register the DataFrame as a SQL temporary view: source
Displaying top 10 rows: 


DataFrame[vendorID: string, tpepPickupDateTime: timestamp, tpepDropoffDateTime: timestamp, passengerCount: int, tripDistance: double, puLocationId: string, doLocationId: string, startLon: double, startLat: double, endLon: double, endLat: double, rateCodeId: int, storeAndFwdFlag: string, paymentType: string, fareAmount: double, extra: double, mtaTax: double, improvementSurcharge: string, tipAmount: double, tollsAmount: double, totalAmount: double, puYear: int, puMonth: int]

In [4]:
df.show(10) # get first 10 rows of data

+--------+-------------------+-------------------+--------------+------------+------------+------------+----------+---------+----------+---------+----------+---------------+-----------+----------+-----+------+--------------------+---------+-----------+-----------+------+-------+
|vendorID| tpepPickupDateTime|tpepDropoffDateTime|passengerCount|tripDistance|puLocationId|doLocationId|  startLon| startLat|    endLon|   endLat|rateCodeId|storeAndFwdFlag|paymentType|fareAmount|extra|mtaTax|improvementSurcharge|tipAmount|tollsAmount|totalAmount|puYear|puMonth|
+--------+-------------------+-------------------+--------------+------------+------------+------------+----------+---------+----------+---------+----------+---------------+-----------+----------+-----+------+--------------------+---------+-----------+-----------+------+-------+
|     CMT|2012-02-29 23:53:14|2012-03-01 00:00:43|             1|         2.1|        null|        null|-73.980494|40.730601|-73.983532|40.752311|         1|   

In [5]:
# smaller dataset bc data is big. Will use the truncated df for the project
df_truncated = df.filter((col("puYear") >= 2017) & (col("puYear") <= 2018))

In [6]:
# view the smaller subset
df_truncated.show(10) 

+--------+-------------------+-------------------+--------------+------------+------------+------------+--------+--------+------+------+----------+---------------+-----------+----------+-----+------+--------------------+---------+-----------+-----------+------+-------+
|vendorID| tpepPickupDateTime|tpepDropoffDateTime|passengerCount|tripDistance|puLocationId|doLocationId|startLon|startLat|endLon|endLat|rateCodeId|storeAndFwdFlag|paymentType|fareAmount|extra|mtaTax|improvementSurcharge|tipAmount|tollsAmount|totalAmount|puYear|puMonth|
+--------+-------------------+-------------------+--------------+------------+------------+------------+--------+--------+------+------+----------+---------------+-----------+----------+-----+------+--------------------+---------+-----------+-----------+------+-------+
|       1|2017-03-02 07:59:01|2017-03-02 08:12:48|             1|         1.4|         238|         236|    null|    null|  null|  null|         1|              N|          1|       9.5|  0.

In [7]:
# count the number of rows in the truncated df
row_count = df_truncated.count()
print("Number of rows in filtered DF:", row_count)  

Number of rows in filtered DF: 216300320


In [8]:
# print schema to inspect data types
df_truncated.printSchema()

root
 |-- vendorID: string (nullable = true)
 |-- tpepPickupDateTime: timestamp (nullable = true)
 |-- tpepDropoffDateTime: timestamp (nullable = true)
 |-- passengerCount: integer (nullable = true)
 |-- tripDistance: double (nullable = true)
 |-- puLocationId: string (nullable = true)
 |-- doLocationId: string (nullable = true)
 |-- startLon: double (nullable = true)
 |-- startLat: double (nullable = true)
 |-- endLon: double (nullable = true)
 |-- endLat: double (nullable = true)
 |-- rateCodeId: integer (nullable = true)
 |-- storeAndFwdFlag: string (nullable = true)
 |-- paymentType: string (nullable = true)
 |-- fareAmount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mtaTax: double (nullable = true)
 |-- improvementSurcharge: string (nullable = true)
 |-- tipAmount: double (nullable = true)
 |-- tollsAmount: double (nullable = true)
 |-- totalAmount: double (nullable = true)
 |-- puYear: integer (nullable = true)
 |-- puMonth: integer (nullable = true)



In [ ]:
# save the df to a CSV file in GCS
bucket_name = 'final-bucketv6' # change based on bucket name created
df_truncated.write.csv(f"gs://{bucket_name}/notebooks/jupyter/Data/")

# This will create a Data folder in GSC and will partion the data into 54 different CSV files due to size. 

# 1. Explain data (i.e.., simple exploratory analysis of various fields, such as the semantic as well the intrinsic meaning of ranges, null values, categorical/numerical, mean/std.dev to normalize and/or scale inputs). Identify any missing or corrupt (i.e., outlier) data.

Data Dictonary. Can also be found at https://learn.microsoft.com/en-us/azure/open-datasets/dataset-taxi-yellow?tabs=pyspark#azure-databricks

The dictonary is organized by the following:
variable name, data type, number of unique values, description (optional)
- doLocationId, string, 265 unique values, TLC Taxi Zone in which the taximeter was disengaged.
- endLat, double, 961,994 unique values
- endLon, double, 1,144,935 unique values
- extra, double, 877 unique values, Miscellaneous extras and surcharges. Currently, this only includes the 0.50 and 1 rush hour and overnight charges. (so only values it contains are 0.5 and 1.0)
- fareAmount, double, 18,935 unique values, The time-and-distance fare calculated by the meter.
- improvementSurcharge, string, 60 unique values, 0.30 improvement surcharge assessed trips at the flag drop. The improvement surcharge began being levied in 2015.
- mtaTax, double, 360 unique values, $0.50 MTA tax that is automatically triggered based on the metered rate in use.
- passengerCount, int, 64 unique values, The number of passengers in the vehicle. This is a driver-entered value.
- paymentType, string, 6,282 unique values, A numeric code signifying how the passenger paid for the trip. 1= Credit card; 2= Cash; 3= No charge; 4= Dispute; 5= Unknown; 6= Voided trip.
- puLocationId, string, 266 unique values, TLC Taxi Zone in which the taximeter was engaged.
- puMonth, int, 12 unique values
- puYear, int, 29 unique values
- rateCodeId, int, 56 unique values, The final rate code in effect at the end of the trip. 1= Standard rate; 2= JFK; 3= Newark; 4= Nassau or Westchester; 5= Negotiated fare; 6= Group ride.
- startLat, double, 833,016 unique values
- startLon, double, 957,428 unique value
- storeAndFwdFlag, string, 8 unique values, This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, also known as “store and forward,” because the vehicle did not have a connection to the server. Y= store and forward trip; N= not a store and forward trip.
- tipAmount, double, 12,121 unique values, This field is automatically populated for credit card tips. Cash tips are not included.
- tollsAmount, double, 6,634 unique values, Total amount of all tolls paid in trip.
- totalAmount, double, 39,707 unique values, The total amount charged to passengers. Does not include cash tips.
- tpepDropoffDateTime, timestamp, 290,185,010 unique values, The date and time when the meter was disengaged.
- tpepPickupDateTime, timestamp, 289,948,585 unique values, The date and time when the meter was engaged.
- tripDistance, double, 14,003 unique values, The elapsed trip distance in miles reported by the taximeter.
- vendorID, string, 7 unique values, A code indicating the TPEP provider that provided the record. 1= Creative Mobile Technologies, LLC; 2= VeriFone Inc.
- vendorID, int, 2 unique values, A code indicating the LPEP provider that provided the record. 1= Creative Mobile Technologies, LLC; 2= VeriFone Inc.

In [11]:
from pyspark.sql.functions import sum

In [12]:
# count missing values for each column
missing_values_counts = df_truncated.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_truncated.columns])

missing_values_counts.show()

# we can see the following colums have missing values: startLon, startLat, endLon, and endLat

+--------+------------------+-------------------+--------------+------------+------------+------------+---------+---------+---------+---------+----------+---------------+-----------+----------+-----+------+--------------------+---------+-----------+-----------+------+-------+
|vendorID|tpepPickupDateTime|tpepDropoffDateTime|passengerCount|tripDistance|puLocationId|doLocationId| startLon| startLat|   endLon|   endLat|rateCodeId|storeAndFwdFlag|paymentType|fareAmount|extra|mtaTax|improvementSurcharge|tipAmount|tollsAmount|totalAmount|puYear|puMonth|
+--------+------------------+-------------------+--------------+------------+------------+------------+---------+---------+---------+---------+----------+---------------+-----------+----------+-----+------+--------------------+---------+-----------+-----------+------+-------+
|       0|                 0|                  0|             0|           0|           0|           0|216300320|216300320|216300320|216300320|         0|              0

We can see the following columns have missing values: startLon, startLat, endLon, and endLat. Since these variables (latitude and longitude) are not crucial to our project we will not need to handle this missing data.

## Basic EDA

In [14]:
from pyspark.sql.functions import year, month

In [15]:
# Count of tax rides per month

# group by year and month then count the number of rides for each time period
rides_count_per_month = df_truncated.groupBy(year("tpepPickupDateTime").alias("puYear"), 
                                             month("tpepPickupDateTime").alias("puMonth")).count()

rides_count_per_month.show()

+------+-------+--------+
|puYear|puMonth|   count|
+------+-------+--------+
|  2017|      3|10294628|
|  2017|      8| 8422197|
|  2017|     10| 9768740|
|  2018|     10| 8821138|
|  2018|      1| 8760120|
|  2018|      3| 9429450|
|  2018|      8| 7849041|
|  2017|      7| 8588486|
|  2018|      5| 9224095|
|  2017|     12| 9508274|
|  2017|      9| 8945575|
|  2017|      4|10046188|
|  2018|      9| 8039933|
|  2018|      7| 7849585|
|  2018|     12| 8172811|
|  2018|     11| 8145749|
|  2018|      6| 8713709|
|  2017|      2| 9168825|
|  2017|     11| 9284777|
|  2017|      5|10102124|
+------+-------+--------+
only showing top 20 rows



In [ ]:
# order by year and month in descending order
rides_count_per_month = rides_count_per_month.orderBy("puYear", "puMonth", ascending=False)

rides_count_per_month.show(24)

+------+-------+--------+
|puYear|puMonth|   count|
+------+-------+--------+
|  2018|     12| 8172811|
|  2018|     11| 8145749|
|  2018|     10| 8821138|
|  2018|      9| 8039933|
|  2018|      8| 7849041|
|  2018|      7| 7849585|
|  2018|      6| 8713709|
|  2018|      5| 9224095|
|  2018|      4| 9305295|
|  2018|      3| 9429450|
|  2018|      2| 8492461|
|  2018|      1| 8760120|
|  2017|     12| 9508274|
|  2017|     11| 9284777|
|  2017|     10| 9768740|
|  2017|      9| 8945575|
|  2017|      8| 8422197|
|  2017|      7| 8588486|
|  2017|      6| 9656993|
|  2017|      5|10102124|
|  2017|      4|10046188|
|  2017|      3|10294628|
|  2017|      2| 9168825|
|  2017|      1| 9710126|
+------+-------+--------+



In [ ]:
# create a bar chart for ride counts per month,year to visually see the data. This takes a while to load. 
# Did not run
import matplotlib.pyplot as plt

# extracting data to plot
years = rides_count_per_month.select("puYear").rdd.flatMap(lambda x: x).collect()
months = rides_count_per_month.select("puMonth").rdd.flatMap(lambda x: x).collect()
counts = rides_count_per_month.select("count").rdd.flatMap(lambda x: x).collect()

plt.figure(figsize=(10, 6))
plt.bar(range(len(counts)), counts, color='skyblue')
plt.xlabel('Month-Year', fontsize=14)
plt.ylabel('Number of Rides', fontsize=14)
plt.title('Number of Taxi Rides per Month-Year', fontsize=16)
plt.xticks(range(len(counts)), [f'{m}-{y}' for m, y in zip(months, years)], rotation=45)
plt.tight_layout()
plt.show()

<Figure size 1000x600 with 1 Axes>

In [19]:
# count of tax rides per zone by month

# group by year, month, and puLocationId and then count the number of rides for each group
rides_count_per_zone_month = df_truncated.groupBy(year("tpepPickupDateTime").alias("puYear"), 
                                                  month("tpepPickupDateTime").alias("puMonth"), 
                                                  "puLocationId").count()

# order by count in descending order to see highest count first
rides_count_per_zone_month = rides_count_per_zone_month.orderBy("count", ascending=False)

rides_count_per_zone_month.show(24)

+------+-------+------------+------+
|puYear|puMonth|puLocationId| count|
+------+-------+------------+------+
|  2017|      5|         237|420877|
|  2018|      5|         237|407246|
|  2017|     10|         237|405086|
|  2017|     12|         237|401288|
|  2017|     11|         237|398611|
|  2018|     10|         237|393569|
|  2017|      4|         237|390697|
|  2018|      4|         237|390523|
|  2017|      5|         236|381377|
|  2017|      3|         237|380997|
|  2017|      1|         237|380663|
|  2017|      6|         237|379701|
|  2017|      3|         161|377452|
|  2017|     10|         161|376038|
|  2018|      3|         237|371555|
|  2018|      4|         161|369959|
|  2017|      5|         161|369772|
|  2017|      4|         161|366820|
|  2018|      3|         161|366648|
|  2017|     12|         236|366262|
|  2017|      1|         236|363649|
|  2018|     11|         237|363062|
|  2018|      5|         236|361821|
|  2017|     11|         161|361249|
+

# 2. Define the outcome (i.e., the evaluation metric and the target) precisely, including mathematical formulas.

Evaluation Metric: Since this is a demand forecasting problem, metrics like RMSE (Root Mean Square Error) or MAPE (Mean Absolute Percentage Error) could be used to measure the accuracy of predictions against actual demand.

Target Variable: The target variable is the count of taxi trips per time period (hourly/daily) for each NYC taxi zone.

In [ ]:
# pseudo-code example for calculating RMSE (Root Mean Square Error) and MAPE (Mean Absolute Percentage Error) for 
# a time series model

import numpy as np

# Root Mean Square Error (RMSE) function
def calculate_rmse(actual_values, predicted_values):
    squared_errors = (actual_values - predicted_values) ** 2
    mean_squared_error = np.mean(squared_errors)
    rmse = np.sqrt(mean_squared_error)
    return rmse

# Mean Absolute Percentage Error (MAPE) function
def calculate_mape(actual_values, predicted_values):
    absolute_percentage_errors = np.abs((actual_values - predicted_values) / actual_values)
    mean_absolute_percentage_error = np.mean(absolute_percentage_errors)
    return mean_absolute_percentage_error

# 3. How do you ingest the data files and represent them efficiently? (think about different file formats)


One way to ingest and represent data files is using Parquet. Parquet is a columnar storage format that is efficient 
for large-scale data files tailored for analytical purposes. This efficiency stems from its compression techniques 
and nested data organization. Parquet also excels in the querying process. Using tools like Apache Spark,
which integrates with Parquet, ensures smooth data reading and processing operations.

After conducting additional analysis, our team has chosen to process the data using PySpark DataFrames (as shown in the code chunks above). We imported our data into a folder in Google Cloud Storage (GCS). Initially, we attempted to write the data to a Parquet file and then read it back in, given that we initially read from a Parquet file. However, this approach proved to be iterative and less efficient.

# 4. Summarized relevant datasets. Describe what tables to join. Describe the workflow for achieving the joins (what are the keys, type of join). Steps to deal with potential missing values


In our project, we operate with a single master dataset, eliminating the necessity for table joins. To address potential missing values within this dataset, we can employ one of two strategies. The first involves removing rows containing missing data. Alternatively, we utilize imputation techniques, which entail filling in missing values with estimated substitutes. Simple imputation methods include replacing missing values with the mean, median, or mode of the respective feature. More advanced techniques include regression imputation, k-nearest neighbors (KNN) imputation, and predictive modeling approaches like decision trees or random forests which offer viable alternatives for handling missing data. Since the missing data we have is not needed for our time series analysis we do not need to deal with missing values. 

# 5. Checkpoint the data after joining to avoid wasting time and resources!

Not needed since we are not joining multiple datasets.

# 6. Summarize how to split the data train/validation/test - making sure no leaks occur; for example: normalize your data using the training statistics.

Splitting data for our time series analysis requires careful consideration to maintain temporal order. Randomly shuffling the data can lead to data leakage so we will split the data based on dates like so:

Training: 1/2017 - 2/2018

Validation: 3/2018 - 6/2018

Testing: 7/2018 - 12/2018

# 7. Credit assignment plan: describe a plan on how the work will be distributed across the team members and how credit will be assigned. 

The assignment questions were evenly distributed among team members. Once individuals finished their assigned questions, we implemented a rotation system where each member reviewed and provided feedback on the answers of their peers, ensuring editing where necessary. In addition to this plan for future milestones we will also implement code reviews. 

# 8. STRETCH Goal for this milestone: code up some or all of the above tasks in rank order of items 1, 2, 3, 4, and 5.

1 basic eda 

2 pseudo-code for evaluation metrics 

3 read in data into pyspark df in the begining of this notebook

4 not related to our project since no joins were needed

5 not related to our project since no joins were needed